In [1]:
import pandas as pd
df_yn = pd.read_csv("C:/medici_class/00.프로젝트/최종프로젝트 임시파일/yeonnam_list.csv")
df_yn

,사업장명,검색결과 가게명,도로명우편번호,검색어,검색어2
0,연남술비,suulbi,3985,03985 연남술비,03985 suulbi
1,유메오뎅,Yume Fish Cake Yeonnam,3982,03982 유메오뎅,03982 Yume Fish Cake Yeonnam
2,헤비메탈,Heavy Metal,3981,03981 헤비메탈,03981 Heavy Metal
3,문일리(moonily),MOONILY PIZZA HOUSE,3974,03974 문일리(moonily),03974 MOONILY PIZZA HOUSE
4,스필아웃,Spill Out (Pizza & Bar),3982,03982 스필아웃,03982 Spill Out (Pizza & Bar)
...,...,...,...,...,...
586,신안동장,신안동장,3986,03986 신안동장,03986 신안동장
587,바다회사랑,바다회사랑,3990,03990 바다회사랑,03990 바다회사랑
588,연남제비,연남제비,3982,03982 연남제비,03982 연남제비
589,유키모찌 연남점,유키모찌 연남점,3982,03982 유키모찌 연남점,03982 유키모찌 연남점


In [3]:
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller
import re

# 드라이버 자동 설치
chromedriver_autoinstaller.install()

# DataFrame에서 검색어 가져오기
search_keywords = df_yn['검색어2'].dropna().unique()
all_data = []

# 드라이버 설정 (한 번만 실행)
options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # 필요시 활성화
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('lang=ko_KR')
options.add_argument("window-size=1024,768")

driver = webdriver.Chrome(options=options)
driver.get("https://www.google.co.kr/maps/?hl=ko")
time.sleep(random.uniform(3, 5))

for idx, keyword in enumerate(search_keywords):
    print(f"[{idx + 1}/{len(search_keywords)}] '{keyword}' 리뷰 수집 중...")

    try:
        # 검색창 찾아서 새 검색어 입력
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "searchboxinput"))
        )
        search_box.clear()
        search_box.send_keys(keyword)
        search_box.send_keys(Keys.ENTER)
        time.sleep(random.uniform(4, 6))

        # 리뷰 탭 클릭
        try:
            tab_buttons = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'button[role="tab"]'))
            )

            review_tab = None
            for btn in tab_buttons:
                try:
                    if "리뷰" in btn.text:
                        review_tab = btn
                        break
                except:
                    continue

            if review_tab:
                driver.execute_script("arguments[0].click();", review_tab)
                time.sleep(5)
            else:
                print(f" - [리뷰탭 없음] {keyword}")
                continue
        except Exception as e:
            print(f" - [리뷰탭 클릭 오류] {keyword}: {e}")
            continue

        # 스크롤 영역
        try:
            scrollable_div = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.m6QErb.DxyBCb.kA9KIf.dS8AEf'))
            )
        except Exception as e:
            print(f" - [스크롤 영역 없음] {keyword}: {e}")
            continue

        prev_review_count = 0
        scroll_attempt = 0

        while True:
            reviews = driver.find_elements(By.CSS_SELECTOR, 'div.jftiEf.fontBodyMedium')
            curr_count = len(reviews)

            if curr_count == prev_review_count:
                scroll_attempt += 1
                if scroll_attempt >= 3:
                    break
            else:
                scroll_attempt = 0
                prev_review_count = curr_count

            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
            time.sleep(random.uniform(1.2, 2.0))

        print(f" - 총 {len(reviews)}개의 리뷰 수집됨.")

        for review in reviews:
            try:
                username = review.find_element(By.CLASS_NAME, 'd4r55 ').text
                profile_link = review.find_element(By.CLASS_NAME, 'WEBjve').get_attribute('data-href')
                try:
                    user_info = review.find_element(By.CLASS_NAME, 'RfnDt').text
                except:
                    user_info = ""
                rating = review.find_element(By.CLASS_NAME, 'kvMYJc').get_attribute("aria-label")
                date = review.find_element(By.CLASS_NAME, 'rsqaWe').text
                try:
                    text = review.find_element(By.CLASS_NAME, 'wiI7pd').text
                except:
                    text = ""

                # 이미지 링크 추출
                image_links = []
                image_buttons = review.find_elements(By.CSS_SELECTOR, 'button.Tya61d')
                for btn in image_buttons:
                    style_attr = btn.get_attribute("style")
                    match = re.search(r'url\("?(https://[^"]*googleusercontent\.com[^"]*)"?\)', style_attr)
                    if match:
                        image_links.append(match.group(1))
                image_links_str = ", ".join(image_links)

                all_data.append({
                    '가게명': keyword,
                    '작성자': username,
                    '작성자 프로필 링크': profile_link,
                    '작성자 정보': user_info,
                    '별점': rating,
                    '리뷰 내용': text,
                    '작성일': date,
                    '이미지 링크': image_links_str
                })
            except Exception as e:
                print(f"  [리뷰 항목 오류] {e}")

    except Exception as e:
        print(f"[검색 오류] {keyword}: {e}")

    # 다음 키워드로 넘어가기 전 약간의 대기
    time.sleep(random.uniform(2, 4))

# 드라이버 종료
driver.quit()

# 결과 저장
df_result = pd.DataFrame(all_data)
df_result.to_csv("gmaps_sel3.csv", index=False, encoding="utf-8-sig")
print("전체 리뷰 수집 완료 및 저장 완료!")

[1/583] '03985 suulbi' 리뷰 수집 중...
 - 총 24개의 리뷰 수집됨.
[2/583] '03982 Yume Fish Cake Yeonnam' 리뷰 수집 중...
 - 총 10개의 리뷰 수집됨.
[3/583] '03981 Heavy Metal' 리뷰 수집 중...
 - 총 15개의 리뷰 수집됨.
[4/583] '03974 MOONILY PIZZA HOUSE' 리뷰 수집 중...
 - 총 21개의 리뷰 수집됨.
[5/583] '03982 Spill Out (Pizza & Bar)' 리뷰 수집 중...
 - 총 136개의 리뷰 수집됨.
[6/583] '03987 Groovy' 리뷰 수집 중...
 - 총 3개의 리뷰 수집됨.
[7/583] '03981 GWEEK' 리뷰 수집 중...
 - [리뷰탭 클릭 오류] 03981 GWEEK: Message: 

[8/583] '03982 화설 연남점 l Hongdae restaurant korean bbq HWASEOL Hongdae Main Store' 리뷰 수집 중...
 - 총 377개의 리뷰 수집됨.
[9/583] '03984 FIESTA7 COFFEE YEONNAM' 리뷰 수집 중...
 - 총 3개의 리뷰 수집됨.
[10/583] '03982 카페 무심 연남(MOOSIM COFFEE CLUB/ MUSIM)' 리뷰 수집 중...
 - 총 3개의 리뷰 수집됨.
[11/583] '03983 Jay&Jason R' 리뷰 수집 중...
 - 총 32개의 리뷰 수집됨.
[12/583] '03982 서양면옥 연남점 / Western Noodle House Yeonnam Branch / 西洋面屋 延南店 / 西洋?屋 延南店' 리뷰 수집 중...
 - 총 214개의 리뷰 수집됨.
[13/583] '03982 연남동 맛집 장끼전 | Korean Restaurant Jangkki Jeon' 리뷰 수집 중...
 - 총 105개의 리뷰 수집됨.
[14/583] '03983 마사비스 카페 연남점 (Mercerbis d

In [2]:
import pandas as pd

In [4]:
# 사업장명 열 추가
g_sel = pd.read_csv("C:/medici_class/00.프로젝트/최종프로젝트 임시파일/gmaps_sel3.csv")
df_yn = pd.read_csv("C:/medici_class/00.프로젝트/최종프로젝트 임시파일/yeonnam_list.csv")

# '가게명'과 '검색어2'가 같은 경우 '사업장명' 가져오기
result = pd.merge(g_sel, df_yn[['검색어2', '사업장명']], how='left', left_on='가게명', right_on='검색어2')

# 불필요한 '검색어2' 열 제거 (선택 사항)
result.drop(columns=['검색어2'], inplace=True)

# 결과 확인
result

,가게명,작성자,작성자 프로필 링크,작성자 정보,별점,리뷰 내용,작성일,이미지 링크,사업장명
0,03985 suulbi,윤지혜,https://www.google.com/maps/contrib/1068785664...,지역 가이드 · 리뷰 12개 · 사진 18장,별표 5개,"남친이랑 연남동 데이트하러 갔다가 발견한 맛집,\r\n가게 분위기도 정말 좋고 저희...",2달 전,https://lh3.googleusercontent.com/geougc-cs/AB...,연남술비
1,03985 suulbi,w J,https://www.google.com/maps/contrib/1175431211...,리뷰 1개 · 사진 2장,별표 5개,"연남 길거리 걷다가 분위기가 좋아서 들어가본 곳인데, 실내 분위기가 더 좋았고 안주...",5달 전,https://lh3.googleusercontent.com/geougc-cs/AB...,연남술비
2,03985 suulbi,신정균 (Sergio),https://www.google.com/maps/contrib/1092095905...,지역 가이드 · 리뷰 14개 · 사진 7장,별표 5개,오랜만에 와이프랑 연남동에서 데이트 하다가 발견한 연남동술집!! 인테리어 분위기가 ...,2달 전,https://lh3.googleusercontent.com/geougc-cs/AB...,연남술비
3,03985 suulbi,아율이네육아일기,https://www.google.com/maps/contrib/1145126506...,지역 가이드 · 리뷰 20개 · 사진 9장,별표 5개,분위기도 좋고 음식도 깔끔해요!\r\n데이트하기 딱 좋은 가게에요ㅎㅎ\r\n스테이크...,2달 전,https://lh3.googleusercontent.com/geougc-cs/AB...,연남술비
4,03985 suulbi,dd dd,https://www.google.com/maps/contrib/1064777297...,리뷰 3개,별표 5개,주말 저녁에 오랜만에 친구들이랑 모임 겸 해서 연남동 술비에 다녀왔어요!! SNS에...,4주 전,NaN,연남술비
...,...,...,...,...,...,...,...,...,...
38058,03982 유키모찌 연남점,Hyo-Seop Shim,https://www.google.com/maps/contrib/1043641344...,지역 가이드 · 리뷰 32개 · 사진 151장,별표 4개,NaN,1년 전,NaN,유키모찌 연남점
38059,03982 뭉텅 연남점,호빵공장,https://www.google.com/maps/contrib/1023673874...,"지역 가이드 · 리뷰 144개 · 사진 3,856장",별표 5개,연트럴파크 맛집 뭉텅 연남점을 찾았습니다. 연남동 고기집 뭉텅은 홍대입구역 3번 출...,11달 전,https://lh3.googleusercontent.com/geougc-cs/AB...,뭉텅 연남점
38060,03982 뭉텅 연남점,Junwon Choi,https://www.google.com/maps/contrib/1131549655...,리뷰 6개 · 사진 24장,별표 5개,본점보다맛있어요! 뭉텅여기서 처음가고 너무맛있어서 본점갔는데 연남점이 훨씬고기가 큰...,7달 전,NaN,뭉텅 연남점
38061,03982 뭉텅 연남점,하릴없이,https://www.google.com/maps/contrib/1162235992...,리뷰 1개,별표 5개,오랜만에 맛있는고기집 찾았어요 고기찾아먹진않는타입인데 가끔땡길때 있잖아유?\r\n사...,1년 전,NaN,뭉텅 연남점


In [5]:
result.columns

Index(['가게명', '작성자', '작성자 프로필 링크', '작성자 정보', '별점', '리뷰 내용', '작성일', '이미지 링크',
       '사업장명'],
      dtype='object')

In [6]:
result = result.drop(['가게명'], axis=1)
result

,작성자,작성자 프로필 링크,작성자 정보,별점,리뷰 내용,작성일,이미지 링크,사업장명
0,윤지혜,https://www.google.com/maps/contrib/1068785664...,지역 가이드 · 리뷰 12개 · 사진 18장,별표 5개,"남친이랑 연남동 데이트하러 갔다가 발견한 맛집,\r\n가게 분위기도 정말 좋고 저희...",2달 전,https://lh3.googleusercontent.com/geougc-cs/AB...,연남술비
1,w J,https://www.google.com/maps/contrib/1175431211...,리뷰 1개 · 사진 2장,별표 5개,"연남 길거리 걷다가 분위기가 좋아서 들어가본 곳인데, 실내 분위기가 더 좋았고 안주...",5달 전,https://lh3.googleusercontent.com/geougc-cs/AB...,연남술비
2,신정균 (Sergio),https://www.google.com/maps/contrib/1092095905...,지역 가이드 · 리뷰 14개 · 사진 7장,별표 5개,오랜만에 와이프랑 연남동에서 데이트 하다가 발견한 연남동술집!! 인테리어 분위기가 ...,2달 전,https://lh3.googleusercontent.com/geougc-cs/AB...,연남술비
3,아율이네육아일기,https://www.google.com/maps/contrib/1145126506...,지역 가이드 · 리뷰 20개 · 사진 9장,별표 5개,분위기도 좋고 음식도 깔끔해요!\r\n데이트하기 딱 좋은 가게에요ㅎㅎ\r\n스테이크...,2달 전,https://lh3.googleusercontent.com/geougc-cs/AB...,연남술비
4,dd dd,https://www.google.com/maps/contrib/1064777297...,리뷰 3개,별표 5개,주말 저녁에 오랜만에 친구들이랑 모임 겸 해서 연남동 술비에 다녀왔어요!! SNS에...,4주 전,NaN,연남술비
...,...,...,...,...,...,...,...,...
38058,Hyo-Seop Shim,https://www.google.com/maps/contrib/1043641344...,지역 가이드 · 리뷰 32개 · 사진 151장,별표 4개,NaN,1년 전,NaN,유키모찌 연남점
38059,호빵공장,https://www.google.com/maps/contrib/1023673874...,"지역 가이드 · 리뷰 144개 · 사진 3,856장",별표 5개,연트럴파크 맛집 뭉텅 연남점을 찾았습니다. 연남동 고기집 뭉텅은 홍대입구역 3번 출...,11달 전,https://lh3.googleusercontent.com/geougc-cs/AB...,뭉텅 연남점
38060,Junwon Choi,https://www.google.com/maps/contrib/1131549655...,리뷰 6개 · 사진 24장,별표 5개,본점보다맛있어요! 뭉텅여기서 처음가고 너무맛있어서 본점갔는데 연남점이 훨씬고기가 큰...,7달 전,NaN,뭉텅 연남점
38061,하릴없이,https://www.google.com/maps/contrib/1162235992...,리뷰 1개,별표 5개,오랜만에 맛있는고기집 찾았어요 고기찾아먹진않는타입인데 가끔땡길때 있잖아유?\r\n사...,1년 전,NaN,뭉텅 연남점


In [7]:
# 원하는 순서대로 열 나열
new_order = ['사업장명', '작성일', '작성자', '작성자 프로필 링크', '작성자 정보', '별점', '리뷰 내용', '이미지 링크']

# 순서대로 재정렬
result = result[new_order]

# 확인
result


,사업장명,작성일,작성자,작성자 프로필 링크,작성자 정보,별점,리뷰 내용,이미지 링크
0,연남술비,2달 전,윤지혜,https://www.google.com/maps/contrib/1068785664...,지역 가이드 · 리뷰 12개 · 사진 18장,별표 5개,"남친이랑 연남동 데이트하러 갔다가 발견한 맛집,\r\n가게 분위기도 정말 좋고 저희...",https://lh3.googleusercontent.com/geougc-cs/AB...
1,연남술비,5달 전,w J,https://www.google.com/maps/contrib/1175431211...,리뷰 1개 · 사진 2장,별표 5개,"연남 길거리 걷다가 분위기가 좋아서 들어가본 곳인데, 실내 분위기가 더 좋았고 안주...",https://lh3.googleusercontent.com/geougc-cs/AB...
2,연남술비,2달 전,신정균 (Sergio),https://www.google.com/maps/contrib/1092095905...,지역 가이드 · 리뷰 14개 · 사진 7장,별표 5개,오랜만에 와이프랑 연남동에서 데이트 하다가 발견한 연남동술집!! 인테리어 분위기가 ...,https://lh3.googleusercontent.com/geougc-cs/AB...
3,연남술비,2달 전,아율이네육아일기,https://www.google.com/maps/contrib/1145126506...,지역 가이드 · 리뷰 20개 · 사진 9장,별표 5개,분위기도 좋고 음식도 깔끔해요!\r\n데이트하기 딱 좋은 가게에요ㅎㅎ\r\n스테이크...,https://lh3.googleusercontent.com/geougc-cs/AB...
4,연남술비,4주 전,dd dd,https://www.google.com/maps/contrib/1064777297...,리뷰 3개,별표 5개,주말 저녁에 오랜만에 친구들이랑 모임 겸 해서 연남동 술비에 다녀왔어요!! SNS에...,NaN
...,...,...,...,...,...,...,...,...
38058,유키모찌 연남점,1년 전,Hyo-Seop Shim,https://www.google.com/maps/contrib/1043641344...,지역 가이드 · 리뷰 32개 · 사진 151장,별표 4개,NaN,NaN
38059,뭉텅 연남점,11달 전,호빵공장,https://www.google.com/maps/contrib/1023673874...,"지역 가이드 · 리뷰 144개 · 사진 3,856장",별표 5개,연트럴파크 맛집 뭉텅 연남점을 찾았습니다. 연남동 고기집 뭉텅은 홍대입구역 3번 출...,https://lh3.googleusercontent.com/geougc-cs/AB...
38060,뭉텅 연남점,7달 전,Junwon Choi,https://www.google.com/maps/contrib/1131549655...,리뷰 6개 · 사진 24장,별표 5개,본점보다맛있어요! 뭉텅여기서 처음가고 너무맛있어서 본점갔는데 연남점이 훨씬고기가 큰...,NaN
38061,뭉텅 연남점,1년 전,하릴없이,https://www.google.com/maps/contrib/1162235992...,리뷰 1개,별표 5개,오랜만에 맛있는고기집 찾았어요 고기찾아먹진않는타입인데 가끔땡길때 있잖아유?\r\n사...,NaN


In [8]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38063 entries, 0 to 38062
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   사업장명        38063 non-null  object
 1   작성일         38063 non-null  object
 2   작성자         38057 non-null  object
 3   작성자 프로필 링크  38063 non-null  object
 4   작성자 정보      36945 non-null  object
 5   별점          38063 non-null  object
 6   리뷰 내용       29127 non-null  object
 7   이미지 링크      15874 non-null  object
dtypes: object(8)
memory usage: 2.3+ MB


In [9]:
result.to_csv("gmaps_reviews_selenium_re.csv", index=False, encoding="utf-8-sig")